# structure gemini putputs with pydantic

In [1]:
from google import genai
client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents = """List a few asian soups recipes, a yummy description and list the ingredients
    give me fields of
    recipe_name: str
    description: str
    ingredients: list[str]

    not in markdown format
    """
)

print(response.text)

recipe_name: Spicy Thai Tom Yum Goong
description: A vibrant and aromatic Thai hot and sour soup that awakens your taste buds with its fiery chili kick, refreshing lemongrass, galangal, and kaffir lime leaves, all swimming with succulent shrimp and mushrooms. It's a symphony of tangy, spicy, and savory flavors that truly brightens any meal and leaves you craving more.
ingredients:
- Shrimp, peeled and deveined
- Mushrooms (straw or button)
- Lemongrass, bruised and sliced
- Galangal, sliced
- Kaffir lime leaves, torn
- Fish sauce
- Lime juice
- Thai bird's eye chilies, crushed (or chili paste)
- Cherry tomatoes, halved
- Cilantro, chopped
- Chicken or vegetable broth
- Evaporated milk or coconut milk (optional, for creamy version)

recipe_name: Savory Japanese Shoyu Ramen
description: Dive into a comforting bowl of Japanese ramen, featuring a deep, umami-rich soy sauce-based broth that's both hearty and incredibly satisfying. Topped with tender slices of chashu pork, a perfectly soft-b

In [ ]:
from pydantic import BaseModel
class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]

client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents = "List a few asian soups recipes, a yummy description and list the ingredients",
    config= {"response_mime_type": "application/json", "response_schema": list[Recipe]} # nu får man json data istället, mer robust
    )


print(response.text)


[{"recipe_name":"Tom Yum Goong","description":"A vibrant and aromatic Thai hot and sour soup, famous for its bold flavors. It's a delightful balance of spicy chili, tangy lime, savory fish sauce, and fresh herbs, typically served with plump shrimp and mushrooms.","ingredients":["Shrimp","Lemongrass","Galangal","Kaffir lime leaves","Lime juice","Fish sauce","Chili paste","Mushrooms","Cilantro","Tomatoes","Chicken or vegetable broth"]},
{"recipe_name":"Miso Soup","description":"A quintessential Japanese comfort food, this savory and light soup is made from dashi (a flavorful broth) and miso paste. It's often accompanied by small cubes of silken tofu, wakame seaweed, and thinly sliced scallions, offering a delicate umami experience.","ingredients":["Miso paste","Dashi broth","Tofu","Wakame seaweed","Scallions"]},
{"recipe_name":"Pho Bo","description":"A beloved Vietnamese noodle soup renowned for its incredibly fragrant and complex broth, simmered for hours with star anise, cinnamon, and 

In [6]:
recipes = response.parsed
recipes, len(recipes)

([Recipe(recipe_name='Tom Yum Goong', description="A vibrant and aromatic Thai hot and sour soup, famous for its bold flavors. It's a delightful balance of spicy chili, tangy lime, savory fish sauce, and fresh herbs, typically served with plump shrimp and mushrooms.", ingredients=['Shrimp', 'Lemongrass', 'Galangal', 'Kaffir lime leaves', 'Lime juice', 'Fish sauce', 'Chili paste', 'Mushrooms', 'Cilantro', 'Tomatoes', 'Chicken or vegetable broth']),
  Recipe(recipe_name='Miso Soup', description="A quintessential Japanese comfort food, this savory and light soup is made from dashi (a flavorful broth) and miso paste. It's often accompanied by small cubes of silken tofu, wakame seaweed, and thinly sliced scallions, offering a delicate umami experience.", ingredients=['Miso paste', 'Dashi broth', 'Tofu', 'Wakame seaweed', 'Scallions']),
  Recipe(recipe_name='Pho Bo', description='A beloved Vietnamese noodle soup renowned for its incredibly fragrant and complex broth, simmered for hours with 

In [7]:
recipes[0].description

"A vibrant and aromatic Thai hot and sour soup, famous for its bold flavors. It's a delightful balance of spicy chili, tangy lime, savory fish sauce, and fresh herbs, typically served with plump shrimp and mushrooms."

## Simulate house prices

In [8]:
from typing import Literal

class Home(BaseModel):
    price: int
    mothly_fee: float
    living_area: float
    number_rooms: int
    type: Literal["apartment", "house"]
    address: str

repsonse = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= """
    List 50 apartments and houses in Sweden with their price, monthly fee, living area, number of rooms, type if it is apartment or house.
    All currencies should be in SEK. Data should be simulation of swedish housing.
""",
    config= {"response_mime_type": "application/json", "response_schema": list[Home]}
)

In [11]:
homes = repsonse.parsed
len(homes)

50

In [15]:
homes[0].__dict__

{'price': 5500000,
 'mothly_fee': 4327.4,
 'living_area': 95.5,
 'number_rooms': 3,
 'type': 'apartment',
 'address': 'Ringvägen 89, Göteborg'}

In [13]:
import pandas as pd

homes

[Home(price=5500000, mothly_fee=4327.4, living_area=95.5, number_rooms=3, type='apartment', address='Ringvägen 89, Göteborg'),
 Home(price=7100000, mothly_fee=5374.8, living_area=119.4, number_rooms=4, type='apartment', address='Skogsvägen 62, Uppsala'),
 Home(price=5700000, mothly_fee=5866.5, living_area=130.4, number_rooms=4, type='house', address='Vasgatan 13, Jönköping'),
 Home(price=2900000, mothly_fee=4043.9, living_area=99.5, number_rooms=3, type='house', address='Tallvägen 70, Linköping'),
 Home(price=6600000, mothly_fee=6021.9, living_area=133.8, number_rooms=4, type='house', address='Björkstigen 97, Malmö'),
 Home(price=3800000, mothly_fee=3863.6, living_area=95.1, number_rooms=3, type='house', address='Kyrkvägen 20, Örebro'),
 Home(price=3000000, mothly_fee=2642.5, living_area=67.6, number_rooms=3, type='apartment', address='Skogsallén 87A, Linköping'),
 Home(price=4600000, mothly_fee=4100.9, living_area=101.9, number_rooms=4, type='apartment', address='Kyrkplatsen 13, Norrk

In [ ]:
pd.DataFrame(homes) # ser förjävlig ut

,0,1,2,3,4,5
0,"(price, 5500000)","(mothly_fee, 4327.4)","(living_area, 95.5)","(number_rooms, 3)","(type, apartment)","(address, Ringvägen 89, Göteborg)"
1,"(price, 7100000)","(mothly_fee, 5374.8)","(living_area, 119.4)","(number_rooms, 4)","(type, apartment)","(address, Skogsvägen 62, Uppsala)"
2,"(price, 5700000)","(mothly_fee, 5866.5)","(living_area, 130.4)","(number_rooms, 4)","(type, house)","(address, Vasgatan 13, Jönköping)"
3,"(price, 2900000)","(mothly_fee, 4043.9)","(living_area, 99.5)","(number_rooms, 3)","(type, house)","(address, Tallvägen 70, Linköping)"
4,"(price, 6600000)","(mothly_fee, 6021.9)","(living_area, 133.8)","(number_rooms, 4)","(type, house)","(address, Björkstigen 97, Malmö)"
5,"(price, 3800000)","(mothly_fee, 3863.6)","(living_area, 95.1)","(number_rooms, 3)","(type, house)","(address, Kyrkvägen 20, Örebro)"
6,"(price, 3000000)","(mothly_fee, 2642.5)","(living_area, 67.6)","(number_rooms, 3)","(type, apartment)","(address, Skogsallén 87A, Linköping)"
7,"(price, 4600000)","(mothly_fee, 4100.9)","(living_area, 101.9)","(number_rooms, 4)","(type, apartment)","(address, Kyrkplatsen 13, Norrköping)"
8,"(price, 2400000)","(mothly_fee, 2074.8)","(living_area, 50.8)","(number_rooms, 2)","(type, apartment)","(address, Tallstigen 69, Helsingborg)"
9,"(price, 4300000)","(mothly_fee, 4160.7)","(living_area, 100.3)","(number_rooms, 4)","(type, house)","(address, Stora gränd 71, Uppsala)"


In [18]:
df = pd.DataFrame([home.__dict__ for home in homes])

In [20]:
cheap_houses = df.query("price < 5000000 and type == 'house'")

cheap_houses

,price,mothly_fee,living_area,number_rooms,type,address
3,2900000,4043.9,99.5,3,house,"Tallvägen 70, Linköping"
5,3800000,3863.6,95.1,3,house,"Kyrkvägen 20, Örebro"
9,4300000,4160.7,100.3,4,house,"Stora gränd 71, Uppsala"
18,4700000,4306.4,106.7,4,house,"Ringallén 5, Norrköping"
19,4600000,4465.7,110.1,4,house,"Kyrkallén 12, Malmö"
29,3800000,3877.8,96.7,3,house,"Vasallén 81, Linköping"
33,4200000,4109.9,100.1,4,house,"Stora gatan 21, Linköping"
40,3700000,3757.2,92.0,3,house,"Vasbacken 44, Jönköping"
44,4400000,4142.1,102.7,4,house,"Kyrkstigen 18, Uppsala"
47,3200000,3337.8,81.8,3,house,"Tallbacken 6, Malmö"


In [23]:
cheap_houses.to_csv("cheaphouses.csv", index=False)